<h1> NOTEBOOK TO CREATE DLC PROJECT, EXTRACT FRAMES & LABEL DATA </h1>

make sure you are running this notebook in properly setup conda enviroment. We don't want to use system wide conda
environments because some operations may require elevated privileges, and other users could break our environment.
Getting a dlc environment working is somewhat difficult on windows, and requires manual management of versions / dependencies.
A working environment is loacted at `F:\MysoreData\nbk\mouseVideoAnalysis\dlc_env_conda`, and can be activated using:

`conda activate "F:\MysoreData\nbk\mouseVideoAnalysis\dlc_env_conda"`

And then relaunching this notebook using the jupyter installation in that env.

In [ ]:
#Errors here mean there are issues with the enviroment dependencies or PATH configuration
import deeplabcut
import os
from random import sample
task = 'box1_cam2'
experimenter = 'spencerloggia'

Get all videos from the provided video source directory. The source dir should have subdirectories for each session,
which in turn contain snippets for each trial, in .avi format. 
**!!In the below cell, make sure the `cam_id` variable is set to the correct camera for this network!!**

In [ ]:
# Change Directory Below For Different Mice
BASE_DIR = r'F:\MysoreData\nbk\mouseVideoAnalysis\Box1\video_data\mitg12'

videodirs = os.listdir(BASE_DIR)
video = []
cam_id = 'camera-1'
for videodir in sample(videodirs, 22):
    videos = os.listdir(os.path.join(BASE_DIR, videodir))
    videos = [path for path in videos if cam_id in path and 'ground' not in path and ('.avi' in path)]
    #print(videos)
    try:
        for vid in sample(videos, 10):
            video.append(os.path.join(BASE_DIR, videodir, vid))  
    except ValueError:
        print("skipping " + str(videodir))
print(video)
num_videos = len(video)

Generate the deeplabcut project, change working directory or else it is created in current directory.
On windows, since symlinks are not supported, we must set `copy_videos=True`. On linux we can get lighter weight
projects and speed up this step by setting `copy_videos=False`.
The output is the path to the config file.

In [ ]:
path_config_file=deeplabcut.create_new_project(task,experimenter,video, working_directory='F:\\MysoreData\\nbk\\mouseVideoAnalysis\\Box1\\cam2\\', copy_videos=True)
print(path_config_file)

At this point take a moment to edit the config.yaml file in the new project directory.
- Set the num2pick variable can be set to change number labeled frames per video. Keep in mind that num2pick will set the number of frames to label for _each_ video clip.
- edit batch_size to change number of labelled images in each set.
- Set the crop dimension of each image to some size that focuses on the mask. I recommend the cropped dimension to be not much greater than 500x500 for performance reasons.
- Set the start and stop parameter to choose frames from a specific video region. start=0 and stop=1 is the full 3 second video. Since the mouse nose pokes at second 1, I recommend using start=.29, stop=.75
- finally, make sure that there are only three bodyparts - bodypart1, bodypart2, and bodypart3. Also make sure that the skeleton parameter is set such as to draw a triangle between the three bodyparts
<h1> Now we can extract frames!</h1><br>
<p>Since crop=True is set, dlc will automatically extract frames at dimmensions selected in the config file. We could crop each frame in a gui, but that is impractical for this many clips.<p/>
    
**!!!!! From now on make sure that the `path_config_file` variable is always set to the correct location for this network !!!!**

In [ ]:
#load matplotlib so outputs can be displayed 
%matplotlib inline 
path_config_file = r'F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam2\box1_cam1-spencerloggia-2021-04-04\config.yaml'
deeplabcut.extract_frames(path_config_file,'automatic','uniform', userfeedback=False, crop=True)

<h1> Now we label the test and training data! </h1> 
    
Once the labelling gui opens, you may have to expand the bottom menu to make sure all buttons are visible.
There will be a folder with the extracted frames for each video. Go into each and right click to place the current label. Skip a label by choosing the next one on the right hand menu.

After labeling a trial, click 'save', then 'exit', then 'yes', to label the next trial.

For consistency: BP1: left ear, BP2: right ear, BP3: nose (tip)

The ear points should be labeled at the most inward point on each ear, though keep in mind that which point appears most inward
may change as the mouse moves its ears or perspective shifts. Therefore, it's useful to establish specific invariant landmarks on the base of the ear
for labelling purposes.

In [ ]:
%gui wx
%matplotlib inline 
path_config_file = r'F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam2\box1_cam1-spencerloggia-2021-04-04\config.yaml'
deeplabcut.label_frames(path_config_file)
    

The last step is just an automatic check of the labelled data. You should see no errors or something has gone wrong, unless you intentionally didn't label some extracted frames, in which case expect DLC to report them as missing.

In [ ]:
path_config_file = r'F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam2\box1_cam1-spencerloggia-2021-04-04\config.yaml'

deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

<h1> Next Steps: </h1>
<h2> In order to actually train the network, we use another notebook. DLC_GPU notebook in this directory and set the path_config_file variable there to the same thing.

In [ ]:
print(path_config_file)